In [ ]:
import torch

# 检查CUDA是否可用
if torch.cuda.is_available():
    # 获取可用的GPU设备数量
    gpu_count = torch.cuda.device_count()
    print(f"Find {gpu_count} GPU can be used.")

    # 遍历并打印每个GPU设备的名称
    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu_name}")
else:
    print("No GPU can be used.")


In [ ]:
import copy
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
# from bert_score import score
import statistics
from ast import literal_eval
import functools
import json
import os
import random
import wget
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

random.seed(8888)
torch.manual_seed(8888)
random.seed(8888)
np.random.seed(8888)

if torch.cuda.is_available():
    torch.cuda.manual_seed(8888)
    torch.cuda.manual_seed_all(8888)


from tqdm import tqdm

torch.set_grad_enabled(False)
tqdm.pandas()

In [ ]:
torch.cuda.set_device(2)


from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join

# Get CounterFact data for GPT2-xl, from the ROME repository.
#wget.download("https://rome.baulab.info/data/dsets/known_1000.json")


model_dir = "/U_PZL2023ZZ0005/yhhong/transformers"
model_name = "Qwen1.5-7B-Chat" #"chatglm3-6b" #"Llama-2-7b-chat-hf" #"Qwen1.5-7B-Chat"


model = AutoModelForCausalLM.from_pretrained(
    join(model_dir, model_name),
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
);

tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name))
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model.to('cuda');


In [ ]:
def set_act_get_hooks(model, tok_index, mlp=False, mlp_coef=False):
    """
    Works on LLaMA-2, getting activation values from certain positions
    """
    # Make sure that these are not set to True at the same time
    #  so we don't put two different hooks on the same module.
    #assert not (attn is True and attn_out is True)

    for attr in ["activations_"]:
        if not hasattr(model, attr):
            setattr(model, attr, {})

    print('tok_index: ',tok_index)

    def get_activation(name):
        def hook(module, input, output):
            if "m_coef" in name:
                # num_tokens = list(input[0].size())[1]  # (batch, sequence, hidden_state)
                #print('output[0].size(): ',output[0].size())
                #print('input[0].size(): ',input[0].size())
                model.activations_[name] = input[0][:, tok_index].detach()
            elif "m_out" in name:
                model.activations_[name] = output[0][tok_index].detach()

        return hook

    hooks = []
    for i in range(32):
        if mlp_coef is True: #co-effciency
            hooks.append(model.model.layers[i].mlp.down_proj.register_forward_hook(get_activation("m_coef_" + str(i))))
        if mlp is True:
            hooks.append(model.model.layers[i].mlp.register_forward_hook(get_activation("m_out_" + str(i))))

    return hooks

def remove_hooks(hooks):
    for hook in hooks:
        hook.remove()
